## EJERCICIO 1 : Se carga el epositorio de Tiny Towns

In [ ]:
# Clonar el repositorio completo
!git clone https://github.com/Augusto-Rabbia/NLP_TP1.git


import os
os.chdir("NLP_TP1/datos")

# Verificar contenido
!ls

## EJERCICIO 2

### Selección y limpieza de texto

Para este ejercicio se selecciononaron todos les textos en español de la sección información y se concatenaron en un solo texto para que asi sea más extenso.

In [ ]:
# Carga textos en español: review_bgg + review_externa + video
import os
import pandas as pd

carpeta = "/content/NLP_TP1/datos/informacion"

archivos_en = ["review_bgg.txt", "review_externa.txt", "video4.txt"]
texto_total = ""
for archivo in archivos_en:
    with open(os.path.join(carpeta, archivo), "r", encoding="utf-8") as f:
        texto_total += f.read() + "\n"


print(f"Total de caracteres: {len(texto_total)}")

Se hace una limpieza ligera de los datos ya que luego vamos a utilizar S-BERT y al eliminar puntuación o normalizar demasiado, se corre el riesgo de quitarle contexto al modelo.

In [ ]:
import re

def limpiar_texto(texto):
    texto = re.sub(r'\s+', ' ', texto)  # reemplazar múltiples espacios y saltos de línea
    texto = re.sub(r'[^\w\sáéíóúüñÁÉÍÓÚÜÑ.,;:!?]', '', texto)  # eliminar símbolos raros
    texto = texto.replace("*", "")
    texto = re.sub(r"([a-z])([A-Z])", r"\1 \2", texto)
    texto = texto.strip()
    return texto

texto_total_limpio = limpiar_texto(texto_total)

### Segmentación

Se separa el texto por saltos de línea dobles, lo que permite identificar bloques temáticamente coherentes.

Luego, se aplica una segmentación "consciente del contenido" que conserva la estructura semántica de manera eficiente y mantiene el contexto.

Finalmente, se agrupan las oraciones para evitar fragmentos demasiado cortos o ambiguos, lo cual mejora la calidad de los embeddings generados por modelos como S-BERT.


In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
import spacy

# Cargar modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")


In [ ]:
fragmentos = []
for bloque in re.split(r'(?:\n\s*){2,}', texto_total):  #  esto detecta bloques separados por líneas vacías
    doc = nlp(bloque.strip())
    oraciones = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 10]

    # Agrupar de a 2 oraciones
    temp = []
    for i, oracion in enumerate(oraciones):
        temp.append(oracion)
        if len(temp) == 2 or i == len(oraciones) - 1:
            fragmentos.append(" ".join(temp))
            temp = []
# Ver algunos ejemplos
for i, frag in enumerate(fragmentos[:5]):
    print(f"Fragmento {i+1}:\n{frag}\n{'-'*40}")

In [ ]:
# Se comprueba la cantidad de tokens que tienen los fragmentos

import pandas as pd
from sentence_transformers import SentenceTransformer

# Cargar el modelo
modelo = SentenceTransformer("distiluse-base-multilingual-cased-v1")
tokenizer = modelo.tokenizer

# Contar tokens de cada fragmento
token_counts = [len(tokenizer(frag)["input_ids"]) for frag in fragmentos]

# Crear DataFrame
df = pd.DataFrame({
    "Fragmento": fragmentos,
    "N_Tokens": token_counts
})

bins = [0, 10, 20, 30, 40, 50, 60, 80, 100, 128, float("inf")]
labels = [
    "0–10", "11–20", "21–30", "31–40", "41–50",
    "51–60", "61–80", "81–100", "101–128", "129+"
]

# Asignar fragmentos a rangos de tokens
df["Rango_Tokens"] = pd.cut(df["N_Tokens"], bins=bins, labels=labels, right=True)

# Calcular frecuencia por rango
frecuencia = df["Rango_Tokens"].value_counts().sort_index()

# tabla de frecuencias
tabla_frecuencia = frecuencia.reset_index()
tabla_frecuencia.columns = ["Rango de Tokens", "Cantidad de Fragmentos"]
print(tabla_frecuencia)


### Vectorización

In [ ]:
from sentence_transformers import SentenceTransformer, util
from prettytable import PrettyTable

In [ ]:
# Cargamos el modelo preentrenado multilingüe
modelo = SentenceTransformer("distiluse-base-multilingual-cased-v1")
# Codificamos los fragmentos
embeddings = modelo.encode(fragmentos, convert_to_tensor=True)

In [ ]:
# Calculamos las puntuaciones de similitud
puntuaciones_coseno = util.cos_sim(embeddings, embeddings)

# Encontramos las puntuaciones de similitud más altas
pares = []
for i in range(len(puntuaciones_coseno)-1):
    for j in range(i+1, len(puntuaciones_coseno)):
        pares.append({'index': [i, j], 'score': puntuaciones_coseno[i][j]})

# Ordenamos las puntuaciones en orden decreciente
pares = sorted(pares, key=lambda x: x['score'], reverse=True)

# Creamos una tabla para mostrar los resultados
tabla = PrettyTable()
tabla.field_names = ["Oración 1", "Oración 2", "Puntuación de Similitud"]

# Añadimos las filas a la tabla
for par in pares[0:10]:
    i, j = par['index']
    tabla.add_row([fragmentos[i], fragmentos[j], f"{par['score']:.4f}"])


print(tabla)

### Análisis de similitud de texto

In [ ]:
consultas = [
    "es un juego que tine un",
    "cuando se termina el juego",
    "reseña y opiniones del juego",
    "como se desarrolla una partida"
]

consulta_embeddings = modelo.encode(consultas, convert_to_tensor=True)

# Calcular similitudes coseno
resultados = util.cos_sim(consulta_embeddings, embeddings)

# Mostrar los fragmentos más similares por cada consulta
top_k = 2
for i, consulta in enumerate(consultas):
    print(f"\n🔍 Consulta: {consulta}")
    valores_sim, idxs = resultados[i].topk(top_k)
    for score, idx in zip(valores_sim, idxs):
        print(f"\n* Similitud: {score.item():.4f}")
        print(f"{fragmentos[idx]}")

### Métricas de Semejanza

In [ ]:
!pip install python-Levenshtein jellyfish

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import jaccard
from Levenshtein import distance as levenshtein_distance
from sklearn.feature_extraction.text import CountVectorizer
import jellyfish

def dice_similarity(s1, s2):
    set1, set2 = set(s1.split()), set(s2.split())
    if not set1 or not set2:
        return 0.0
    return 2 * len(set1 & set2) / (len(set1) + len(set2))

def buscar_similares(frase_consulta, top_k=3):
    print(f"\n🔍 Frase de búsqueda: {frase_consulta}")

    # Vector S-BERT de la consulta
    vector_consulta = modelo.encode([frase_consulta], convert_to_tensor=True)

    # ---------- MÉTRICA 1: Coseno ----------
    similitudes_coseno = cosine_similarity(vector_consulta, embeddings)[0]
    top_coseno_idx = np.argsort(similitudes_coseno)[-top_k:][::-1]

    print("\n📐 Resultados por *Similitud del coseno*:")
    for idx in top_coseno_idx:
        print(f"  ({similitudes_coseno[idx]:.3f}) {fragmentos[idx][:100]}...")

    # ---------- MÉTRICA 2: Jaccard ----------
    vectorizer = CountVectorizer(binary=True)
    binarized = vectorizer.fit_transform([frase_consulta] + fragmentos)
    jaccard_scores = [
        1 - jaccard(binarized[0].toarray()[0], binarized[i+1].toarray()[0])
        for i in range(len(fragmentos))
    ]
    top_jaccard_idx = np.argsort(jaccard_scores)[-top_k:][::-1]

    print("\n📐 Resultados por *Distancia de Jaccard*:")
    for idx in top_jaccard_idx:
        print(f"  ({jaccard_scores[idx]:.3f}) {fragmentos[idx][:100]}...")

    # ---------- MÉTRICA 3: Dice ----------
    dice_scores = [dice_similarity(frase_consulta, frag) for frag in fragmentos]
    top_dice_idx = np.argsort(dice_scores)[-top_k:][::-1]

    print("\n📐 Resultados por *Similitud de Dice*:")
    for idx in top_dice_idx:
        print(f"  ({dice_scores[idx]:.3f}) {fragmentos[idx][:100]}...")


    return vector_consulta, top_coseno_idx


In [ ]:
vector_consulta, top_coseno_idx = buscar_similares("cuando o como finaliza el juego para un los participantes")

Para realizar la búsqueda semántica entre fragmentos de texto, probamos distintas métricas de Semejanza:

**Similitud del coseno**, que utiliza vectores densos de embeddings para capturar significado más allá de palabras literales.

**Distancia de Jaccard** y **Similitud de Dice**, que comparan palabras presentes en los textos en forma binaria.

Otras métricas como Levenshtein o Jaro-Winkler se descartaron porque miden diferencias a nivel de caracteres y no son adecuadas para evaluar similitud semántica entre textos largos o fragmentados.

**La Similitud del coseno fue la que mejor reflejó la relación semántica entre la consulta y los fragmentos, por lo que se eligió como métrica principal para el análisis.**

### Visualización

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualizar_tsne_3D(embeddings, vector_consulta, fragmentos, top_idx=None):
    import numpy as np

    # Unir embeddings + consulta
    all_vectors = np.vstack([vector_consulta, embeddings])

    # Aplicar t-SNE
    tsne = TSNE(n_components=3, perplexity=30, n_iter=1000, random_state=42, init='pca')
    emb_3d = tsne.fit_transform(all_vectors)

    # Separar los puntos
    consulta_3d = emb_3d[0]
    frag_3d = emb_3d[1:]

    # Graficar
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Fragmentos
    ax.scatter(frag_3d[:, 0], frag_3d[:, 1], frag_3d[:, 2], alpha=0.6, label='Fragmentos')

    # Consulta
    ax.scatter(consulta_3d[0], consulta_3d[1], consulta_3d[2], c='red', s=100, label='Frase de búsqueda')

    # Más similares
    if top_idx is not None and len(top_idx) > 0:
        ax.scatter(frag_3d[top_idx, 0], frag_3d[top_idx, 1], frag_3d[top_idx, 2],
                   c='green', s=60, label='Más similares')

    ax.set_title("Visualización 3D con t-SNE")
    ax.legend()
    plt.show()

    return emb_3d


In [ ]:
emb_3d_tsne = visualizar_tsne_3D(
    embeddings=embeddings,
    vector_consulta=vector_consulta,
    fragmentos=fragmentos,
    top_idx=top_coseno_idx  # o top_jaccard_idx, etc.
)


## EJERCICIO 3

### Cargamos el texto

In [ ]:
#cargamos el texto
ruta = "/content/NLP_TP1/datos/informacion/manual.txt"

with open(ruta, "r", encoding="utf-8") as file:
    texto_manual = file.read()

# Análisis básico
num_caracteres = len(texto_manual)
num_palabras = len(texto_manual.split())

print(f" El texto contiene {num_caracteres} caracteres y {num_palabras} palabras.\n")

In [ ]:
import re

# Limpieza mínima
texto_manual_limpio = re.sub(r'\s+', ' ', texto_manual).strip()

### Segmentación con spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
# spaCy en inglés
nlp = spacy.load("en_core_web_lg")

In [ ]:
# Segmentación del texto
doc_total = nlp(texto_manual_limpio)
fragmentos = [sent.text.strip() for sent in doc_total.sents]

### POS y NER

In [ ]:
!pip install spacy gliner

In [ ]:
from gliner import GLiNER

# GLiNER multilingüe
gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
gliner.eval()

In [ ]:
sustantivos_por_fragmento = []
entidades_por_fragmento = []

for i, frag in enumerate(fragmentos):

    # spaCy: extraer sustantivos
    doc_frag = nlp(frag)
    nouns = [token.text for token in doc_frag if token.pos_ in {"NOUN", "PROPN"}]
    sustantivos_por_fragmento.append(nouns)
    print("Sustantivos detectados (POS):", nouns)

    # GLiNER: entidades personalizadas
    labels = ["person", "book", "location", "date", "character", "board game", "GameComponent",
              "ResourceType","BuildingType","Role", "RuleName"]
    ents = gliner.predict_entities(frag, labels, threshold=0.4)
    entidades_por_fragmento.append(ents)

    for ent in ents:
        print(f" - {ent['text']} => {ent['label']}")


In [ ]:
sustantivos_ner = []

for ents in entidades_por_fragmento:
    # Extraer solo el texto de las entidades (que son los sustantivos o nombres detectados por NER)
    sustantivos_ner.append([ent["text"] for ent in ents])

In [ ]:
print(sustantivos_ner)

### Búsqueda de similitud

#### Vectorización con FastText

In [ ]:
!pip install fasttext

In [ ]:
import fasttext
import fasttext.util
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Descargar modelo en inglés
fasttext.util.download_model('en', if_exists='ignore')
ft_model = fasttext.load_model('cc.en.300.bin')

#### Métricas

In [ ]:
import numpy as np
import Levenshtein
import jellyfish
from sklearn.metrics.pairwise import cosine_similarity as sk_cosine_similarity

# --- Funciones métricas ---

def cosine_similarity(vec1, vec2):
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0.0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def jaccard_distance(str1, str2):
    set1, set2 = set(str1), set(str2)
    inter = set1.intersection(set2)
    union = set1.union(set2)
    return 1 - len(inter) / len(union) if union else 1.0

def dice_similarity(str1, str2):

    def bigrams(s):
        return set([s[i:i+2] for i in range(len(s)-1)]) if len(s) > 1 else set()
    bigr1 = bigrams(str1)
    bigr2 = bigrams(str2)
    inter = bigr1.intersection(bigr2)
    return 2 * len(inter) / (len(bigr1) + len(bigr2)) if (len(bigr1)+len(bigr2)) > 0 else 0.0

def levenshtein_distance(str1, str2):
    return Levenshtein.distance(str1, str2)

def jaro_winkler_similarity(str1, str2):
    return jellyfish.jaro_winkler_similarity(str1, str2)

#### Funciones de busqueda

In [ ]:

# Aplanar la lista de listas de sustantivos filtrados por NER y pasar a minúsculas
sustantivos_filtrados = [noun.lower() for sublist in sustantivos_ner for noun in sublist]

# Quitar duplicados
sustantivos_unicos = list(set(sustantivos_filtrados))


# --- Funciones de búsqueda por cada métrica ---

def buscar_por_coseno(query, lista, ft_model, top_k=3):
    query_vec = ft_model.get_word_vector(query.lower())
    similitudes = []
    for palabra in lista:
        vec = ft_model.get_word_vector(palabra)
        sim = cosine_similarity(query_vec, vec)
        similitudes.append((palabra, sim))
    similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)
    return similitudes[:top_k]

def buscar_por_jaccard(query, lista, top_k=3):
    distancias = []
    for palabra in lista:
        dist = jaccard_distance(query.lower(), palabra)
        distancias.append((palabra, dist))
    distancias = sorted(distancias, key=lambda x: x[1])
    return distancias[:top_k]

def buscar_por_levenshtein(query, lista, top_k=3):
    distancias = []
    for palabra in lista:
        dist = levenshtein_distance(query.lower(), palabra)
        distancias.append((palabra, dist))
    distancias = sorted(distancias, key=lambda x: x[1])
    return distancias[:top_k]

def buscar_por_dice(query, lista, top_k=3):
    similitudes = []
    for palabra in lista:
        sim = dice_similarity(query.lower(), palabra)
        similitudes.append((palabra, sim))
    similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)
    return similitudes[:top_k]

def buscar_por_jaro_winkler(query, lista, top_k=3):
    similitudes = []
    for palabra in lista:
        sim = jaro_winkler_similarity(query.lower(), palabra)
        similitudes.append((palabra, sim))
    similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)
    return similitudes[:top_k]



#### Busqueda de Similitud con distancias

Para una “similitud textual” (si se parecen las palabras de forma escrita) las métricas como Jaccard, Levenshtein, Dice, Jaro-Winkler son adecuadas porque miden la similitud basada en caracteres o tokens. Entre estas métricas, Jaro-Winkler suele ser la más efectiva porque pondera los prefijos comunes y corrige bien errores de tipeo al inicio de las palabras,.

Pero si pensamos en “similitud semántica” (relación de significado o contexto) similitud por coseno es la mejor opción.

In [ ]:
# Función general que muestra todo
def buscar_similitudes(query, palabras, ft_model, top_k=3):
    print(f"\nSimilitud coseno con FastText para '{query}':")
    for palabra, score in buscar_por_coseno(query, palabras, ft_model, top_k):
        print(f"  {palabra}: {score:.3f}")

    print(f"\nSimilitud de Jaccard para '{query}':")
    for palabra, score in buscar_por_jaccard(query, palabras, top_k):
        print(f"  {palabra}: {score:.3f}")

    print(f"\nSimilitud de Levenshtein para '{query}':")
    for palabra, score in buscar_por_levenshtein(query, palabras, top_k):
        print(f"  {palabra}: {score:.3f}")

    print(f"\nSimilitud de Dice para '{query}':")
    for palabra, score in buscar_por_dice(query, palabras, top_k):
        print(f"  {palabra}: {score:.3f}")

    print(f"\nSimilitud de Jaro-Winkler para '{query}':")
    for palabra, score in buscar_por_jaro_winkler(query, palabras, top_k):
        print(f"  {palabra}: {score:.3f}")


In [ ]:
buscar_similitudes("Fyni tonw", sustantivos_unicos, ft_model)

In [ ]:
buscar_similitudes("Construct", sustantivos_unicos, ft_model)

## EJERCICIO 4

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect, DetectorFactory
import os
import shutil
import pandas as pd

DetectorFactory.seed = 0

# Carpeta raíz donde están las carpetas 'estadisticas', 'informacion', 'relaciones'
root_dir = '/content/NLP_TP1/datos'

resultados = []

for carpeta, subcarpetas, archivos in os.walk(root_dir):
    for archivo in archivos:
        ruta = os.path.join(carpeta, archivo)

        if archivo.endswith(('.txt', '.csv')):
            try:
                with open(ruta, 'r', encoding='utf-8', errors='ignore') as f:
                    texto = f.read()
                if texto.strip():
                    idioma = detect(texto)
                else:
                    idioma = 'vacío'
                resultados.append({'archivo': ruta, 'idioma': idioma})
            except Exception as e:
                print(f"Error leyendo {ruta}: {e}")

# Creamos el DataFrame con resultados
df_idiomas = pd.DataFrame(resultados)

print(df_idiomas)

# Copiamos los archivos en carpetas separadas según el idioma
carpeta_destino = '/content/NLP_TP1/datos_separados'

os.makedirs(carpeta_destino, exist_ok=True)

for _, fila in df_idiomas.iterrows():
    archivo = fila['archivo']
    idioma = fila['idioma']

    carpeta_idioma = os.path.join(carpeta_destino, idioma)
    os.makedirs(carpeta_idioma, exist_ok=True)

    nombre_archivo = os.path.basename(archivo)
    destino = os.path.join(carpeta_idioma, nombre_archivo)

    shutil.copy2(archivo, destino)


## EJERCICIO 5

### Análisis de sentimientos (BERT multilingüe)

In [ ]:
# Reemplazá con el nombre correcto de tu archivo
df = pd.read_csv("/content/NLP_TP1/datos/informacion/df_foros_bgg.csv")

# Nos quedamos solo con las Reviews
reviews_df = df[df["Category"] == "Reviews"].copy()
reviews_df.dropna(subset=["Conversation"], inplace=True)
reviews_df.reset_index(drop=True, inplace=True)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# Modelo entrenado en varios idiomas
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Pipeline de análisis de sentimientos
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Función para convertir estrellas a polaridad
def stars_to_label(label):
    stars = int(label.split()[0])
    if stars <= 2:
        return "NEGATIVE"
    elif stars == 3:
        return "NEUTRAL"
    else:
        return "POSITIVE"

# Aplicar análisis a cada review
reviews_df["Sentiment_raw"] = reviews_df["Conversation"].apply(lambda x: nlp(x[:512])[0])  # truncar a 512 tokens
reviews_df["Sentiment"] = reviews_df["Sentiment_raw"].apply(lambda x: stars_to_label(x["label"]))

In [ ]:
reviews_df.to_csv("reviews_con_sentimientos.csv", index=False)

In [ ]:
reviews_df[["Conversation", "Sentiment_raw", "Sentiment"]]

### Sistema de búsqueda semántica + filtro por sentimiento

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar modelo para embeddings
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Crear y almacenar los embeddings
reviews_df["Embeddings"] = embed_model.encode(reviews_df["Conversation"].tolist()).tolist()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def buscar_reviews(query, sentimiento=None, top_n=2):
    # Obtener embedding del query
    query_vec = embed_model.encode([query])

    # Filtrar por sentimiento si se indica
    if sentimiento:
        df_filtrado = reviews_df[reviews_df["Sentiment"] == sentimiento.upper()].copy()
    else:
        df_filtrado = reviews_df.copy()

    # Si no hay reviews que coincidan con el filtro
    if df_filtrado.empty:
        print("No se encontraron reseñas con ese sentimiento.")
        return pd.DataFrame()

    # Asegurar que las columnas de embeddings estén en formato de matriz
    embedding_matrix = np.vstack(df_filtrado["Embeddings"].values)

    # Calcular similitud coseno
    similitudes = cosine_similarity(query_vec, embedding_matrix)[0]

    # Obtener los top_n resultados más similares
    top_idx = np.argsort(similitudes)[-top_n:][::-1]
    resultados = df_filtrado.iloc[top_idx].copy()
    resultados["Similitud"] = similitudes[top_idx]

    return resultados[["Conversation", "Sentiment", "Similitud"]]

In [ ]:
buscar_reviews("Great game components and rules", sentimiento="POSITIVE", top_n=2)

## EJERCICIO 6

###Carga del set de datos (300 preguntas categorizadas)

In [ ]:
df = pd.read_csv('/content/preguntas_tiny_towns (1).csv')

print(df.head(10))

In [ ]:
# Contar cuántas preguntas hay de cada categoría
conteo_categorias = df['Categoría'].value_counts()

print(conteo_categorias)

In [ ]:
df.columns = df.columns.str.strip().str.lower()
df.rename(columns={'pregunta': 'pregunta', 'categoría': 'categoria'}, inplace=True)

### Modelo de Clasificación

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim.utils


X = df['pregunta'].values
y = df['categoría'].values

# División train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# TF-IDF + LogisticRegression

# Vectorizar texto con TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Entrenar LR
lr_tfidf = LogisticRegression(max_iter=1000)
lr_tfidf.fit(X_train_tfidf, y_train)

# Predecir
y_pred_tfidf = lr_tfidf.predict(X_test_tfidf)

# Métricas TF-IDF
print("*TF-IDF + Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print("F1-score:", f1_score(y_test, y_pred_tfidf, average='weighted'))
print("Precision:", precision_score(y_test, y_pred_tfidf, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_tfidf, average='weighted'))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred_tfidf))

# Predicción consulta nueva con TF-IDF
consulta = ["¿quien invento tiny towns?"]
consulta_vec = tfidf_vectorizer.transform(consulta)
print(consulta)
print("Predicción TF-IDF:", lr_tfidf.predict(consulta_vec))


#Doc2Vec + LogisticRegression

# Preparar documentos etiquetados
train_tagged = [TaggedDocument(words=gensim.utils.simple_preprocess(doc), tags=[str(i)]) for i, doc in enumerate(X_train)]

# Entrenar modelo Doc2Vec
doc2vec_model = Doc2Vec(vector_size=50, min_count=2, epochs=40)
doc2vec_model.build_vocab(train_tagged)
doc2vec_model.train(train_tagged, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# Función para vectorizar textos con Doc2Vec
def doc2vec_vectorize(texts):
    return [doc2vec_model.infer_vector(gensim.utils.simple_preprocess(text)) for text in texts]

X_train_d2v = doc2vec_vectorize(X_train)
X_test_d2v = doc2vec_vectorize(X_test)

# Entrenar LR con vectores Doc2Vec
lr_d2v = LogisticRegression(max_iter=1000)
lr_d2v.fit(X_train_d2v, y_train)

# Predecir
y_pred_d2v = lr_d2v.predict(X_test_d2v)

# Métricas Doc2Vec
print("\n*Doc2Vec + Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_d2v))
print("F1-score:", f1_score(y_test, y_pred_d2v, average='weighted'))
print("Precision:", precision_score(y_test, y_pred_d2v, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_d2v, average='weighted'))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred_d2v))

# Predicción consulta nueva con Doc2Vec
consulta_d2v = doc2vec_vectorize(consulta)
print(consulta)
print("Predicción Doc2Vec:", lr_d2v.predict(consulta_d2v))